# QAOA - KNAPSACK PROBLEM

In [1]:
# INDICAR SI ESTÁS EN GOOGLE COLAB O NO (JUPYTER...)
colab = False
if colab:
    !pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np
from itertools import product
import cmath
from collections import defaultdict

In [3]:
# Datos del problema
values = [10, 8, 15, 6, 12, 7, 9, 11]# , 5, 13, 16, 14, 18, 4, 17, 2, 19, 3, 20, 1]
weights = [5, 3, 8, 2, 6, 4, 3, 5]#, 2, 7, 9, 6, 10, 1, 8, 1, 9, 2, 10, 1]
W = 18  # capacidad de la mochila
n = len(values)

# Parámetros QAOA
p = 15  # número de capas QAOA
dev = qml.device("default.qubit", wires=n)

Sean $v_i$ los diferentes valores de los objetos, $w_i$ los diferentes pesos de los objetos, y $W$ el peso total de la mochila. Entonces la función de coste a minimizar (queremos el estado de mínima energía) es el siguiente hamiltoniano

$$
min \ H_c = - \sum_{i=1}^n v_i x_i +A · (\sum_{i=1}^n x_iw_i-W)^2
$$

Donde A es una penalización por no ajustarse al peso máximo. Un requisito de las funciones de coste hamiltonianas es que deben de ser suaves. Por eso aquí también penalizo las soluciones factibles que se alejan mucho del peso máximo. Pero es lo que hay :)

In [ ]:
# Penalización por violar la restricción
A = 100  # mayor penalización => más se respeta la restricción

# Hamiltoniano del problema: maximizar valores, penalizar pesos
def cost_hamiltonian():
    coeffs = []
    observables = []

    # Maximizar valor: -v_i * Z_i
    for i in range(n):
        coeffs.append(0.5 * values[i])
        observables.append(qml.PauliZ(i))
        coeffs.append(-0.5 * values[i])  # para ajustar el sesgo
        observables.append(qml.Identity(0))

    # Penalización por peso > W: A*(w_total - W)^2
    for i in range(n):
        for j in range(n):
            coeffs.append(0.25 * A * weights[i] * weights[j])
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

    for i in range(n):
        coeffs.append(A * weights[i] * W)
        observables.append(qml.PauliZ(i))

    coeffs.append(A * W * W)
    observables.append(qml.Identity(0))

    return qml.Hamiltonian(coeffs, observables)

H_cost = cost_hamiltonian()

Vamos a repasar la función de coste Hamiltoniana y por qué esta implementación

En primer lugar, tener en cuenta que el valor binario $x_i \in \{0, 1 \}$ se puede representar como $x_i = \frac{1-Z_i}{2}$.

El primer término de la función de coste es:

$$
- \sum_{i=1}^n v_ix_i = - \sum_{i=1}^n v_i\frac{1-Z_i}{2} = -\frac{1}{2} \sum_{i=1}^n v_i + \frac{1}{2} \sum_{i=1}^n v_iZ_i
$$

De ahí que en Python hagamos:

```
for i in range(n):
  coeffs.append( 0.5 * values[i])        # → coef. para Z_i
  observables.append(qml.PauliZ(i))

  coeffs.append(-0.5 * values[i])        # → coef. para I
  observables.append(qml.Identity(0))
```

Luego,

$$
A · (\sum_{i=1}^n x_iw_i-W)^2 = A (\sum_i \sum_j w_iw_jx_ix_j -2W \sum_i w_i x_i + W^2)
$$

Primer término del sumatorio: Desarrollamos $x_ix_j$:

$$
x_ix_j = \frac{1-Z_i}{2} \frac{1-Z_j}{2} = \frac{1}{4} (1-Z_i-Z_j+Z_iZ_j)
$$

Por lo que en el sumatorio nos aparecen tres tipos de términos:
- $\frac{1}{4}w_iw_jZ_iZ_j$
- $-\frac{1}{4}w_iw_jZ_i$ ó $-\frac{1}{4}w_iw_jZ_j$
- $\frac{1}{4}w_iw_j$ que es constante

Podríamos incluir todos los términos, pero sólo incluimos el cuadrático (el primero), porque los términos lineales en $Z_i​$ ó $Z_j$ y constantes se añaden después por separado, mientras que los términos $Z_iZ_j$​ son los que dan la dependencia cuadrática y correlaciones.

Y teniendo en cuenta $A$, escribimos en el código

```
for i in range(n):
        for j in range(n):
            coeffs.append(0.25 * A * weights[i] * weights[j])
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))
```

Segundo término del sumatorio: Sustituimos $x_i$:

$$
-2W \sum_i w_i x_i = -2W \sum_i w_i \frac{1-Z_i}{2} = -W \sum_i w_i + W \sum_i w_i Z_i
$$

Podríamos incluir el primer término, pero es una constante, por lo que no resulta necesario. Sólo incluimos el segundo término

```
for i in range(n):
        coeffs.append(A * weights[i] * W))
        observables.append(qml.PauliZ(i))
```

Tercer término del sumatorio: Se añade directamente:

```
coeffs.append(A * W * W)
observables.append(qml.Identity(0))
```

Implementamos capas RX porque el hamiltoniano del mezclador es $H_{mezcla} = \sum_i X_i$. Usamos $X_i$ porque para problemas donde las posibles soluciones son cadenas de bits (cada bit es cero o uno), $X$ nos permite cambiar ceros por unos y viceversa. Al hacer $e^{-i \beta H_{mezcla}} = \prod_i e^{-i \beta X_i}$, nos damos cuenta de que $e^{-i \beta X_i} = RX(2\beta)$

In [ ]:
# Operadores de mezcla: X_i
def mixer_layer(beta):
    for i in range(n):
        qml.RX(-2 * beta, wires=i)

# Operadores de coste: e^{-i gamma H}
def cost_layer(gamma):
    qml.ApproxTimeEvolution(H_cost, -gamma, 1)

# QAOA circuit
@qml.qnode(dev)
def qaoa_circuit(params):
    gammas = params[:p]
    betas = params[p:]

    # Estado inicial: superposición uniforme
    for i in range(n):
        qml.Hadamard(wires=i)

    for i in range(p):
        cost_layer(gammas[i])
        mixer_layer(betas[i])

    return qml.probs(wires=range(n))

# Función de coste para entrenamiento clásico
def objective(params):
    probs = qaoa_circuit(params)
    cost = 0
    for i, bitstring in enumerate(product([0, 1], repeat=n)):
        z = np.array(bitstring)
        total_value = np.dot(z, values)
        total_weight = np.dot(z, weights)
        penalty = A * max(total_weight - W, 0) ** 2
        cost += probs[i] * (penalty - total_value)
    return cost

# Inicialización y optimización
np.random.seed(42)
init_params = np.random.uniform(0, np.pi, 2 * p)
opt = qml.optimize.AdamOptimizer(stepsize=0.1)

steps = 100
params = init_params

for i in range(steps):
    print(i)
    params = opt.step(objective, params)

# Resultados
probs = qaoa_circuit(params)
bitstrings = list(product([0, 1], repeat=n))
sorted_results = sorted(zip(probs, bitstrings), reverse=True)

print("\nResultados más probables:")
for prob, b in sorted_results[:5]:
    total_value = np.dot(b, values)
    total_weight = np.dot(b, weights)
    print(f"{b} -> Valor = {total_value}, Peso = {total_weight}, Probabilidad = {prob:.4f}")

## Iteración a mano

In [ ]:
# Datos del problema
values = [8, 47, 10, 5, 16]
weights = [3, 11, 14, 19, 5]
W = 26  # capacidad de la mochila
n = len(values)

# Parámetros QAOA
p = 10  # número de capas QAOA
dev = qml.device("default.qubit", wires=n)

# Penalización por violar la restricción
A = 100  # mayor penalización => más se respeta la restricción

def cost_hamiltonian(A_1, A_2):
    coeffs = []
    observables = []

    # ----- TÉRMINO DE VALOR -----
    for i in range(n):
        coeffs.append(0.5 * values[i])
        observables.append(qml.PauliZ(i))

    # ----- PENALIZACIÓN LINEAL (-A1 * (sum w_i x_i - W)) -----
    for i in range(n):
        coeffs.append(0.5 * A_1 * weights[i])
        observables.append(qml.PauliZ(i))

    # ----- PENALIZACIÓN CUADRÁTICA A2 * (sum w_i x_i - W)^2 -----
    # Primer término cuadrático: 0.25 * A2 * sum_{i,j} w_i w_j Z_i Z_j
    for i in range(n):
        for j in range(n):
            coeffs.append(0.25 * A_2 * weights[i] * weights[j])
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # Segundo término lineal: -0.5 * A2 * W * sum_i w_i Z_i
    for i in range(n):
        coeffs.append(-0.5 * A_2 * W * weights[i])
        observables.append(qml.PauliZ(i))

    return qml.Hamiltonian(coeffs, observables)

# Operadores de mezcla: X_i
def mixer_layer(beta):
    for i in range(n):
        qml.RX(-2 * beta, wires=i)

# Operadores de coste: e^{-i gamma H}
def cost_layer(gamma, A_1, A_2):
    H_cost = cost_hamiltonian(A_1, A_2)
    qml.ApproxTimeEvolution(H_cost, -gamma, 1)

In [ ]:

# QAOA circuit
gamma = 0.5
beta = 0.5
A_1 = 5
A_2 = 0.5

puertas = 40

@qml.qnode(dev)
def circuit(restantes):

    # Estado inicial: superposición uniforme
    for i in range(n):
        qml.Hadamard(wires=i)

    while restantes>0:

      if restantes > 0:
        cost_layer(gamma, A_1, A_2)
        restantes -= 1

      if restantes > 0:
        mixer_layer(beta)
        restantes -= 1

    return qml.state()

for num_puertas in range(puertas):

  probs = [abs(i)**2 for i in circuit(num_puertas)]
  fases = [cmath.phase(i) for i in circuit(num_puertas)]

  plt.bar(range(len(probs)), probs)
  plt.title(f"Probabilidades con {num_puertas} puertas")
  plt.show()

  plt.bar(range(len(fases)), fases)
  plt.title(f"Fases con {num_puertas} puertas")
  plt.show()

  print("------------------------------------------------------------------------------------")


## Unbalanced Penalization

In [ ]:
# Datos del problema
values = [8, 47, 10, 5, 16]
weights = [3, 11, 14, 19, 5]
W = 26  # capacidad de la mochila
n = len(values)

# Parámetros QAOA
p = 10  # número de capas QAOA
dev = qml.device("default.qubit", wires=n)

In [ ]:
def cost_hamiltonian(A_1, A_2):
    coeffs = []
    observables = []

    # ----- TÉRMINO DE VALOR -----
    for i in range(n):
        coeffs.append(0.5 * values[i])
        observables.append(qml.PauliZ(i))

    # ----- PENALIZACIÓN LINEAL (-A1 * (sum w_i x_i - W)) -----
    for i in range(n):
        coeffs.append(0.5 * A_1 * weights[i])
        observables.append(qml.PauliZ(i))

    # ----- PENALIZACIÓN CUADRÁTICA A2 * (sum w_i x_i - W)^2 -----
    # Primer término cuadrático: 0.25 * A2 * sum_{i,j} w_i w_j Z_i Z_j
    for i in range(n):
        for j in range(n):
            coeffs.append(0.25 * A_2 * weights[i] * weights[j])
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # Segundo término lineal: -0.5 * A2 * W * sum_i w_i Z_i
    for i in range(n):
        coeffs.append(-0.5 * A_2 * W * weights[i])
        observables.append(qml.PauliZ(i))

    return qml.Hamiltonian(coeffs, observables)

In [ ]:
# Operadores de mezcla: X_i
def mixer_layer(beta):
    for i in range(n):
        qml.RX(-2 * beta, wires=i)

# Operadores de coste: e^{-i gamma H}
def cost_layer(gamma, A_1, A_2):
    H_cost = cost_hamiltonian(A_1, A_2)
    qml.ApproxTimeEvolution(H_cost, -gamma, 1)

# QAOA circuit
@qml.qnode(dev)
def qaoa_circuit(params):
    gammas = params[:p]
    betas = params[p:-2]
    A_1 = params[-2]
    A_2 = params[-1]

    # Estado inicial: superposición uniforme
    for i in range(n):
        qml.Hadamard(wires=i)

    for i in range(p):
        cost_layer(gammas[i], A_1, A_2)
        mixer_layer(betas[i])

    return qml.probs(wires=range(n))

# Función de coste para entrenamiento clásico
def objective(params):
    probs = qaoa_circuit(params)
    A_1 = params[-2]
    A_2 = params[-1]
    cost = 0
    for i, bitstring in enumerate(product([0, 1], repeat=n)):
        z = np.array(bitstring)
        total_value = np.dot(z, values)
        total_weight = np.dot(z, weights)
        penalty_1 = A_1 * max(total_weight - W, 0)
        penalty_2 = A_2 * max(total_weight - W, 0) ** 2
        cost += probs[i] * (penalty_2 + penalty_1 - total_value)
    return cost

# Inicialización y optimización
np.random.seed(42)
init_params = np.random.uniform(0, np.pi, 2 * p)

# Penalización por violar la restricción
A_1 = 1 # 0.96
A_2 = 0.05 #0.0371  # mayor penalización => más se respeta la restricción

opt = qml.optimize.AdamOptimizer(stepsize=0.1)
steps = 500
params = qml.math.concatenate([init_params, [A_1, A_2]])

for i in range(steps):
    if i % 10 == 0:
        print(i)
    params = opt.step(objective, params)

# Resultados
probs = qaoa_circuit(params)
bitstrings = list(product([0, 1], repeat=n))
sorted_results = sorted(zip(probs, bitstrings), reverse=True)

print("\nResultados más probables:")
for prob, b in sorted_results[:5]:
    total_value = np.dot(b, values)
    total_weight = np.dot(b, weights)
    print(f"{b} -> Valor = {total_value}, Peso = {total_weight}, Probabilidad = {prob:.4f}")

In [ ]:
print(sorted_results)

dict_valores = dict()
dict_pesos = dict()
for prob, b in sorted_results:
    total_value = np.dot(b, values)
    total_weight = np.dot(b, weights)

    if total_weight <=W:
      if int(total_value) in dict_valores.keys():
          dict_valores[int(total_value)] += prob.item()
      if int(total_weight) in dict_pesos.keys():
          dict_pesos[int(total_weight)] += prob.item()
      else:
          dict_valores[int(total_value)] = prob.item()
          dict_pesos[int(total_weight)] = prob.item()
    """
    if int(total_value) in dict_valores.keys():
          dict_valores[int(total_value)] += prob.item()
    if int(total_weight) in dict_pesos.keys():
        dict_pesos[int(total_weight)] += prob.item()
    else:
        dict_valores[int(total_value)] = prob.item()
        dict_pesos[int(total_weight)] = prob.item()
    """
#print(dict_valores)

fig, ax = plt.subplots()
ax.bar(dict_valores.keys(), dict_valores.values())
ax.set_xlabel('Valor')
ax.set_ylabel('Probabilidad')
ax.set_title('Probabilidad de Valores')
plt.show()

fig, ax = plt.subplots()
ax.bar(dict_pesos.keys(), dict_pesos.values())
ax.set_xlabel('Peso')
ax.set_ylabel('Probabilidad')
ax.set_title('Probabilidad de Pesos')
plt.show()

In [ ]:
print(params)

## Slack Variables

¿Y cuál es el problema de la mochila? Imaginate que tienes una serie de objetos $x_i$ con $i = 1, ..., n$, los cuales tienen cada uno un determinado valor $v_i$ y un determinado peso $w_i$. Tienes una mochila de peso máximo $W$, y tu objetivo es maximizar el valor de los objetos que decidas llevar en la mochila sin pasarte del peso máximo.

Formulemos el problema matemáticamente. Nuestras variables son $x_i \in \{0, 1\} → \text{El objeto no se mete en la mochila o se mete}$, con $i = 1,...,n$, siendo $n$ el número de objetos. Cada objeto tiene asociados un valor $v_i$ y un peso $w_i$. Una constante importante a tener en cuenta es el peso máximo $W$.

Dado todo esto, la función a optimizar es

$$
max \ f(x_1,...x_n) = \sum_{i=1}^n x_i · v_i
$$

sujeto a la restricción

$$
\sum_{i=1}^n x_i · w_i \leq W → \sum_{i=1}^n x_i · w_i - W \leq 0
$$

Vale, ¿pero no habíamos dicho que las restricciones de desigualdad no queríamos ni verlas por aquí? Vamos a hacer lo siguiente para intentar solucionar este problema. Vamos a suponer que los pesos $w_1,...,w_n \in \mathbb{N}$ (y si no, intenta que multiplicando todos por algún número puedan cumplir esto). Podemos recurrir a lo que se llaman **slack variables** o **variables de holgura**.

Una variable de holgura $s$ hace pasar de restricciones de desigualdad a restricciones de igualdad así ($a$ es un vector de coeficientes y $x$ de variables, $c$ es una constante):

$$
a^T x \leq c \rightarrow a^Tx+s=c
$$

Entonces $c$ tomará el valor que tenga que tomar para cumplir la igualdad.

Pero claro, estamos forzados a que las variables de nuestro problema sean $0$ ó $1$... Entonces si el peso máximo es $W$, y consideramos la opción de que ningún objeto esté dentro de la mochila, tendríamos que meter en principio W variables de holgura... Y a poco que $W$ sea grande... Por eso, hacemos lo siguiente para reducir bastante el número de variables de holgura: expresar la diferencia entre el peso que metemos a la mochila y el peso máximo en binario. Entonces el número de variables de holgura que necesitamos es $k = \lfloor log_2 W \rfloor +1$ (por ejemplo, si $W = 17$, necesitamos $k = \lfloor log_2 16 \rfloor + 1 = 4 + 1 = 5$ variables de holgura para poder expresar hasta el número $32$ si quisiéramos). Ahora nuestra restricción pasa a ser

$$
\sum_{i=1}^n w_ix_i + \sum_{j=0}^{k-1} 2^j y_j = W
$$

Por lo que incluyéndola en la función objetivo con una penalización y elevada al cuadrado para penalizar en cualquier caso que no se cumpla la igualdad (pero si el peso introducido en realidad no supera el peso máximo permitido, las variables slack harán que se respete la igualdad), tenemos que **minimizar** (cambiamos el signo de los términos)

$$
min f(x_1,...,x_n,y_1,...,y_k) = -\sum_{i=1}^n v_ix_i + P \bigg( \sum_{i=1}^n w_ix_i + \sum_{j=0}^{k-1} 2^j y_j - W \bigg)^2
$$

Ahora tenemos que hacer el cambio de variable $x_i = \frac{1-z_i}{2}$

El primer sumatorio:

$$
- \sum_{i=1}^n x_i · v_i = -\sum_{i=1}^n \frac{1-z_i}{2} v_i = -\sum_{i=1}^n \frac{1}{2} v_i + \sum_{i=1}^n \frac{1}{2}v_i · z_i
$$

Pero sólo implementamos el segundo término porque el primero es una constante.

Vamos con la segunda parte, que es un poco más peliaguda.

Sea la penalización:

$$
P\left( \sum_{i=1}^{n} w_i x_i + \sum_{j=0}^{k-1} 2^j y_j - W \right)^2
$$

Aplicando el cambio de variables (las variables $y_j$ no dejan de tomar los valores $0$ ó $1$:

$$
x_i = \frac{1 - Z_i}{2}, \quad y_j = \frac{1 - Z_{n+j}}{2}
$$

La expresión se transforma en:

$$
P\left( \frac{1}{2} \left( \sum_{i=1}^{n} w_i - \sum_{i=1}^{n} w_i Z_i \right)
+ \frac{1}{2} \left( \sum_{j=0}^{k-1} 2^j - \sum_{j=0}^{k-1} 2^j Z_{n+j} \right) - W \right)^2
$$

Agrupando:

$$
P\left( C - \frac{1}{2} \left( \sum_{i=1}^{n} w_i Z_i + \sum_{j=0}^{k-1} 2^j Z_{n+j} \right) \right)^2
$$

donde la constante es:

$$
C = \frac{1}{2} \sum_{i=1}^{n} w_i + \frac{1}{2} \sum_{j=0}^{k-1} 2^j - W
$$

Expandimos el cuadrado:

$$
P \left[ C^2 - C \left( \sum_{i=1}^{n} w_i Z_i + \sum_{j=0}^{k-1} 2^j Z_{n+j} \right)
+ \frac{1}{4} \left( \sum_{i<k} 2 w_i w_k Z_i Z_k +
\sum_{j<l} 2 \cdot 2^j \cdot 2^l Z_{n+j} Z_{n+l} +
\sum_{i,j} 2 w_i \cdot 2^j Z_i Z_{n+j} \right) \right]
$$

Despreciando constantes e identidades, el Hamiltoniano penalizador resultante es:

$$
H = \sum_i \alpha_i Z_i + \sum_j \beta_j Z_{n+j}
+ \sum_{i<k} \gamma_{ik} Z_i Z_k + \sum_{j<l} \delta_{jl} Z_{n+j} Z_{n+l}
+ \sum_{i,j} \eta_{ij} Z_i Z_{n+j}
$$

con coeficientes:

$$
\alpha_i = -P C w_i, \quad
\beta_j = -P C \cdot 2^j, \quad
\gamma_{ik} = \frac{P}{2} w_i w_k, \quad
\delta_{jl} = \frac{P}{2} \cdot 2^j \cdot 2^l, \quad
\eta_{ij} = \frac{P}{2} w_i \cdot 2^j
$$


In [ ]:
# Parámetros del problema
v = np.array([1, 2, 3])          # valores de los ítems (n = 3)
w = np.array([3, 4, 2])          # pesos de los ítems (n = 3)
n = len(w)
W = 7                            # capacidad máxima
k = int(np.log2(W)) + 1          # número de bits para slack variable
P = 100                          # penalización

# Cálculo de la constante C
C = 0.5 * np.sum(w) + 0.5 * np.sum([2**j for j in range(k)]) - W
def cost_hamiltonian():

    # Inicializamos listas de coeficientes y operadores
    coeffs = []
    ops = []

    # Valores introducidos en la mochila
    for i in range(n):
        coeffs.append(-0.5*v[i])
        ops.append(qml.PauliZ(i))

    # Penalizaciones
    # Términos lineales en Z_i (items)
    for i in range(n):
        coeffs.append(-P * C * w[i])
        ops.append(qml.PauliZ(i))

    # Términos lineales en Z_{n+j} (slack variables)
    for j in range(k):
        coeffs.append(-P * C * 2**(j))
        ops.append(qml.PauliZ(n + j))

    # Términos cuadráticos Z_i Z_k (items)
    for i in range(n):
        for k_ in range(i+1, n):
            coeffs.append(0.5 * P * w[i] * w[k_])
            ops.append(qml.PauliZ(i) @ qml.PauliZ(k_))

    # Términos cuadráticos Z_{n+j} Z_{n+l} (slack variables)
    for j in range(k):
        for l in range(j+1, k):
            coeffs.append(0.5 * P * 2**(j) * 2**(l))
            ops.append(qml.PauliZ(n + j) @ qml.PauliZ(n + l))

    # Términos mixtos Z_i Z_{n+j}
    for i in range(n):
        for j in range(k):
            coeffs.append(0.5 * P * w[i] * 2**(j))
            ops.append(qml.PauliZ(i) @ qml.PauliZ(n + j))

    # Construcción del Hamiltoniano
    return qml.Hamiltonian(coeffs, ops)

H_cost = cost_hamiltonian()

In [ ]:
def cost_layer(gamma):
    qml.ApproxTimeEvolution(H_cost, gamma, 1)

def mixer_layer(beta):
    for i in range(n):
        qml.RX(beta, wires=i)

In [ ]:
p = 1 # Número de capas
n_wires = n + k # El numero total de variables es las normales junto con las slack
dev = qml.device('default.qubit', wires=n_wires)

@qml.qnode(dev)
def qaoa_circuit(params):
    gammas = params[:p]
    betas = params[p:]

    for i in range(n_wires):
        qml.Hadamard(wires=i)

    for i in range(p):
        cost_layer(gammas[i])
        mixer_layer(betas[i])

    return qml.expval(H_cost) # qml.probs(wires=range(n))

# Función de costo
def cost(params):
    return qaoa_circuit(params)

np.random.seed(42)
init_params = np.random.uniform(0, np.pi, 2 * p)
params = init_params

stepsize = 0.002

opt = qml.optimize.AdamOptimizer(stepsize)
steps = 500

for i in range(steps):
    params = opt.step(cost, params)
    cost_value = cost(params)
    if i % 100 == 0:
        print(f"Step {i + 1}: Cost = {cost_value:.6f}")

In [ ]:
@qml.qnode(dev)
def qaoa_circuit_probs(params):
    gammas = params[:p]
    betas = params[p:]

    for i in range(n_wires):
        qml.Hadamard(wires=i)

    for i in range(p):
        cost_layer(gammas[i])
        mixer_layer(betas[i])

    return qml.probs(wires=range(n_wires))

# Probabilidades
probs_values = qaoa_circuit_probs(params)
bitstrings = list(product([0, 1], repeat=n_wires))

sorted_results = sorted(zip(probs_values, bitstrings), key=lambda x: x[0], reverse=True)

print("\nTop bitstrings (most probable):")
for prob, bits in sorted_results[:10]:
    print(f"{bits}: {prob:.4f}")

# Preparar datos para el gráfico
probabilities = [x[0] for x in sorted_results]
bitstrings_str = [''.join(map(str, x[1])) for x in sorted_results]

fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(bitstrings_str, probabilities)
ax.tick_params(axis='x', rotation=60)
ax.set_title('Probabilidades')
plt.show()

In [ ]:
# Agrupar las probabilidades según los primeros n bits (ítems reales)
grouped_probs = defaultdict(float)

for prob, bits in zip(probs_values, bitstrings):
    item_bits = bits[:n]  # ignoramos slack bits
    key = ''.join(map(str, item_bits))
    grouped_probs[key] += prob  # sumamos probabilidad total para ese conjunto de ítems

# Ordenar los resultados por probabilidad descendente
sorted_grouped = sorted(grouped_probs.items(), key=lambda x: x[1], reverse=True)

# Mostrar el top
print("\nTop soluciones (ignorando slack variables):")
for bits, prob in sorted_grouped[:10]:
    print(f"{bits}: {prob:.4f}")

# Preparar datos para el gráfico
bitstrings_str = [x[0] for x in sorted_grouped]
probabilities = [x[1] for x in sorted_grouped]

# Plot
fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(bitstrings_str, probabilities)
ax.tick_params(axis='x', rotation=60)
ax.set_title('Probabilidades agrupadas por ítems (sin slack variables)')
plt.show()